In [56]:
import scipy.io
import numpy as np

data = scipy.io.loadmat('data.mat')
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'Bias5', 'Bias_avg', 'DR_event_idx', 'K', 'MAPE5', 'MAPE_avg', 'NARE5', 'NARE_avg', 'customer', 'days', 'event_period', 'flag', 'i', 'test5'])


In [61]:
data['customer'][0][0][1][0][0]
# [customer]表示数据集的名称
# 第二个[]表示custmoer底下的内容，由于只有一份，故为0
# 第三个[]表示选取的行数,即为选取哪个用户
# 第四个[0]内容为选取用户的id，[1]表示用户数据
# 第五个[]表示选取结构体的行数,即为单个用户的表示的小时数
# 第六个[]表示列数，即为天数(171)

# 建立存储每名用户的字典
dict_user = {}
num_of_user = len(data['customer'][0])
print('num_of_user:',num_of_user)
num_of_day = len(data['customer'][0][0][1][0])
print("num_of_day:",num_of_day)

# dict_user{}存储每个用户的数据列表的字典
for user_index in range(num_of_user-3000):
    temp_data = []
    for i in range(num_of_day):
        for j in range(24):
            temp_data.append(data['customer'][0][user_index][1][j][i])
    dict_user[user_index] = temp_data

print(np.array(dict_user[0]).shape)

num_of_user: 3637
num_of_day: 171
()


In [58]:
# 计算自相关系数
def ac_calculate(num_list, lag_num=24*7):
    # 计算自协方差
    mean_num_list = np.mean(np.array(num_list))
    auto_conv_total = 0
    for i in range(len(num_list) - lag_num):
        auto_conv_total += (num_list[i] - mean_num_list)*(num_list[i+lag_num]-mean_num_list)
    auto_conv = auto_conv_total/(len(num_list)-lag_num)
    # 计算均方差
    error_total = 0
    for i in range(len(num_list)):
        error_total += (num_list[i] - mean_num_list)**2
    error = error_total/len(num_list)
    return auto_conv/error
# print(a)

In [59]:
from random import shuffle
import time

num_cluster  = 10
# 读取用户数据的序号
X = dict_user.keys()
X = list(X)

# 随机打乱用户序号
shuffle(X)
# print(X)
cluster = [[x] for x in X[:num_cluster]]
print(cluster)

dict_cluster = {}
for i in range(10):
    dict_cluster[i] = dict_user[cluster[i][0]]
print(dict_cluster.keys())

for list_index in range(len(X)-num_cluster):
    T = time.time()
    dict_cluster_temp = dict_cluster
    for dict_index in range(10):
        for i in range(len(dict_user[X[list_index + num_cluster]])):
            dict_cluster_temp[dict_index][i] += dict_user[X[list_index + num_cluster]][i]
    dict_record ={}
    for index in range(num_cluster):
        dict_record[index] = ac_calculate(dict_cluster_temp[index]) - ac_calculate(dict_cluster[index])
    max_ac_increase = dict_record[0]
    max_num = 0
    for i in range(10-1):
        if dict_record[i+1] > max_ac_increase:
            max_ac_increase = dict_record[i+1]
            max_num = i + 1
    for i in range(len(dict_user[X[list_index + num_cluster]])):
        dict_cluster[max_num][i] += dict_user[X[list_index + num_cluster]][i]
    print(time.time()-T)

[[515], [327], [84], [338], [601], [196], [313], [2], [444], [468]]
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
0.0729982852935791
0.07332086563110352
0.07607507705688477
0.082427978515625
0.08111929893493652
0.08488106727600098
0.08291316032409668
0.07200098037719727
0.07081890106201172
0.07281708717346191
0.07208919525146484
0.0700235366821289
0.07188868522644043
0.0722799301147461
0.07209348678588867
0.07492995262145996
0.0700993537902832
0.07589173316955566
0.0720834732055664
0.09574532508850098
0.09001708030700684
0.0760037899017334
0.06564617156982422
0.06886625289916992
0.07402563095092773
0.07018375396728516
0.06899809837341309
0.07500195503234863
0.07109975814819336
0.07120823860168457
0.0678706169128418
0.07099723815917969
0.06929731369018555
0.06940054893493652
0.07118821144104004
0.07108259201049805
0.07127189636230469
0.07750797271728516
0.06450724601745605
0.06606340408325195
0.06401181221008301
0.06653022766113281
0.06663012504577637
0.06587576866149902
0.06555414199829102

In [60]:
# test
dict_user[cluster[0][0]]
a = ac_calculate(dict_user[cluster[0][0]])
print(a)
print(dict_cluster[4])


0.9491512952151204
[200.6804999999999, 148.71699999999976, 128.55349999999996, 115.65999999999997, 119.5970000000001, 127.85599999999998, 161.48799999999974, 213.88150000000005, 264.74850000000004, 280.63649999999984, 271.10899999999975, 304.80649999999986, 345.2304999999999, 322.993, 284.48350000000016, 285.85050000000007, 335.21750000000003, 419.94100000000003, 417.1909999999996, 393.24299999999977, 387.3124999999999, 395.7254999999993, 383.33549999999997, 309.85500000000036, 210.51050000000004, 151.45999999999998, 124.22200000000014, 110.50899999999994, 115.02750000000003, 120.67799999999991, 151.2605, 231.3865, 269.09599999999995, 283.4465000000002, 297.47250000000014, 281.98900000000003, 315.41750000000013, 315.569, 256.22950000000014, 259.8469999999999, 299.9675000000003, 409.75849999999974, 382.23200000000065, 337.4174999999997, 331.1245000000006, 322.89499999999975, 368.95800000000037, 301.3520000000001, 205.22700000000012, 147.87050000000016, 121.45450000000001, 109.0639999999